### Quiz 3: Solution CHEME 3130 Fall 2020

#### Part a) Derivation (see SVN Example 7.5). Highlights
We were given:

\begin{equation}
\mu_{JT} = -\frac{1}{C_{P}}\left(\frac{\partial{H}}{\partial{P}}\right)_{T}
\end{equation}

Lets tackle the enthalpy term. Starting from FPR for enthalpy, $dH$ = $TdS+VdP$, divide through by dP and then hold T constant:

\begin{equation}
\left(\frac{\partial{H}}{\partial{P}}\right)_{T} = T\left(\frac{\partial{S}}{\partial{P}}\right)_{T}+V
\end{equation}

However, we know that entropy term can be re-written as:
\begin{equation}
\left(\frac{\partial{S}}{\partial{P}}\right)_{T} = -\left(\frac{\partial{V}}{\partial{T}}\right)_{P}
\end{equation}

which gives:

\begin{equation}
\left(\frac{\partial{H}}{\partial{P}}\right)_{T} = V - T\left(\frac{\partial{V}}{\partial{T}}\right)_{P}
\end{equation}

To get this in terms of the compressibility Z, substitute V = ZRT/P into the enthalpy expression:

\begin{equation}
\left(\frac{\partial{H}}{\partial{P}}\right)_{T} = \frac{ZRT}{P} - T\left[\frac{ZR}{P}+\frac{RT}{P}\left(\frac{\partial{Z}}{\partial{T}}\right)_{P}\right]
\end{equation}

But the first two terms cancel, leaving the desired final answer:

\begin{equation}
\mu_{JT} = \frac{RT^{2}}{PC_{P}}\left(\frac{\partial{Z}}{\partial{T}}\right)_{P}
\end{equation}


#### Part b) What is the $\mu_{JT}$ for an ideal gas?
There a couple of ways to do this. First, we know that:

\begin{equation}
\mu_{JT} = -\frac{1}{C_{P}}\left(\frac{\partial{H}}{\partial{P}}\right)_{T}
\end{equation}

but

\begin{equation}
\left(\frac{\partial{H}}{\partial{P}}\right)_{T} = V - T\left(\frac{\partial{V}}{\partial{T}}\right)_{P}
\end{equation}

Thus, we can use the ideal gas law to compute the derivative term, and then plug in the volume from the IGL:

\begin{equation}
\left(\frac{\partial{H}}{\partial{P}}\right)_{T} = \frac{RT}{P} - T\left(\frac{R}{P}\right)
\end{equation}

all terms cancel. Alternatively, we know that Z = 1 for an ideal gas, thus any derivative of Z will vanish.

#### Part c) Estimate the $\mu_{JT}$ for Methane at P = 3 MPa and T = 400K (Numerical Strategy)

There are a couple of ways to do this calculation. The challenge is that V(T,P) appears in the compressibility $Z$ expression, but we do not have an explicit form for it as van der Waals (vdW) is a Cubic Equation of State (CEOS) this we cannnot solve for the volume explicitly. However, we can compute the compressibility derivative numerically e.g.,

\begin{equation}
\left(\frac{\partial{Z}}{\partial{T}}\right)_{P}\simeq\frac{Z\left(T+\Delta{T},P\right)-Z\left(T,P\right)}{\Delta{T}}
\end{equation}

This is a [finite difference approximation](https://en.wikipedia.org/wiki/Finite_difference) of the derivative; in this particular case, a forward difference which comes directly from the definition of a [derivative in calculus](https://en.wikipedia.org/wiki/Derivative). However, to make this work, we'll need to numerically solve the vdW equation for $V\left(T,P\right)$ and $V\left(T+\Delta{T},P\right)$.
But we can poise that calculation as a minimization problem:

\begin{equation}
\min_{V} \left[\left(P-\frac{RT}{V-b}+\frac{a}{V^{2}}\right)^2+\gamma\Bigl(\max(0,-V)+\max(0,-(V-b)\Bigr)\right]
\end{equation}

In [23]:
# Setup paths and environment -
# YOU"LL NEED TO CHANGE THIS TO WHERE YOU RUN LOCALLY -
path_to_project_files = "/Users/jeffreyvarner/Desktop/julia_work/CHEME-3130-F2020"
cd(path_to_project_files)

# include files -
include("Include.jl");

 Activating environment at `~/Desktop/julia_work/CHEME-3130-F2020/Project.toml`


In [19]:
# Setup the volume estimation -
Pc = 4.61   # MPa
Vc = 0.0986 # L/mol
Tc = 190.6  # K
R = 8.313e-3 # L MPa mol^-1 K^-1
ω = 0.253

# build a working fluid model -
methane_working_fluid_model = CCBESingleComponentWorkingFluid(Tc,Pc,Vc,ω; R=R);

# Build a vdW equation of state from that working model -
vdW_eos_model = buildVanDerWaalsEquationOfState(methane_working_fluid_model);

In [52]:
# what is my 𝝙T?
𝝙T = 5.0 # K
TBase = 400; # K

In [64]:
# estimate the volume at two T's -
# base case: P = 3 MPa and T = 400K
P1 = 3.0 # MPa
T1 = TBase - 𝝙T  # K
volume_1 = volume(vdW_eos_model,P1,T1; volumeGuess=1.0)

1.0687317947614845

In [54]:
# lets look at T + ΔT -
P1 = 3.0 # MPa => remember pressure is constant in the derivative -
T2 = TBase + 𝝙T # K => lets use a 5K diff
volume_2 = volume(vdW_eos_model,P1,T2; volumeGuess=1.0)

1.0982127953026493

In [55]:
# Now that we have the volume estimates, we can compute the derivative -
a = vdW_eos_model.a
b = vdW_eos_model.b
Z_vdW_1 = ((volume_1)/(volume_1 - b)) - a/(R*T1*volume_1)
Z_vdW_2 = ((volume_2)/(volume_2 - b)) - a/(R*T2*volume_2)
(Z_vdW_1,Z_vdW_2)

(0.9764164970138045, 0.9785768789303102)

In [56]:
# compute the derivative using the Z_vdW terms -
dZdT_c_P_vdW = (Z_vdW_2-Z_vdW_1)/(2*𝝙T)

0.00021603819165056938

In [65]:
# Compute \mu_{JT} -
Cp = 40.63*(0.001) # MPa-L/mol-K
mu_JT = ((R*TBase^2)/(Cp*P1))*dZdT_c_P_vdW

2.357437672906631

##### Part c) Alternative strategy 1: Instead of solving for the vdW volume numerically - could we use the IGL volume (and still compute the derivative numerically)?

In [58]:
# compute the volumes w/the IGL -
volume_1_IGL = (R*T1)/(P1)
volume_2_IGL = (R*T2)/(P1)
(volume_1_IGL,volume_2_IGL)

(1.0945449999999999, 1.1222549999999998)

In [59]:
# Use the IGL volumes in Z_vdW?
Z_vdW_1_IGL = ((volume_1_IGL)/(volume_1_IGL - b)) - a/(R*T1*volume_1_IGL)
Z_vdW_2_IGL = ((volume_2_IGL)/(volume_2_IGL - b)) - a/(R*T2*volume_2_IGL)
(Z_vdW_1_IGL,Z_vdW_2_IGL)

(0.9769323238617957, 0.9790011100108751)

In [60]:
# compute the dZdT_const_p w/the IGL volumes -
dZdT_c_P_IGL = (Z_vdW_2_IGL-Z_vdW_1_IGL)/(2*𝝙T)

0.00020687861490793492

In [61]:
# Compute \mu_{JT} -
Cp = 40.63*(0.001) # MPa-L/mol-K
mu_JT = ((R*TBase^2)/(Cp*P1))*dZdT_c_P_IGL

2.2574871451041596

#### Part c) alternative strategy 2: Ott and Boerio-Goates 2000
Replace the V(T,P) in the vdW compressibility with the Ideal Gas Law, then analytically compute the derivative which gives (Eqn 3.90):

\begin{equation}
\mu_{JT}\simeq\frac{1}{Cp}\left[\frac{2a}{RT}-b-\frac{3abP}{R^{2}T^{2}}\right]
\end{equation}

In [63]:
# lets eval the Ott expression -
mu_JT_Ott = (1/Cp)*((2*a)/(R*TBase) - b - (3*a*b*P1)/((R^2)*(TBase^2)))

2.1458747300141843